In [1]:
import pandas as pd
import numpy as np


In [2]:
#uvoz podataka
#sezona 23/24
df_23_4 = pd.read_csv("ligue-1-2023.csv")
#sezona 24/25
df_24_5 = pd.read_csv("ligue-1-2024.csv")
#sezona 25/6
df_25_6 = pd.read_csv("ligue-1-2025.csv")

In [3]:
## Ukidanje Duplih Vrijednosti za Nazive Klubova
#za sezonu 24 5
df_24_5["Home Team"] = df_24_5["Home Team"].replace("Stade DE Reims", 'Stade de Reims')
df_24_5["Away Team"] = df_24_5["Away Team"].replace("Stade DE Reims", 'Stade de Reims')

#za sezonu 25 6
df_23_4["Home Team"] = df_23_4["Home Team"].replace("Havre AC", 'Havre Athletic Club')
df_23_4["Away Team"] = df_23_4["Away Team"].replace("Havre AC", 'Havre Athletic Club')

## Obavezno!!!

In [4]:
#odrediti duzinu sezone
df_25_6 = df_25_6.head(117)
df_25_6.tail()

,Match Number,Round Number,Date,Location,Home Team,Away Team,Result
112,109,13,23/11/2025 14:00,Stade de l'Abbé-Deschamps,AJ Auxerre,Olympique Lyonnais,0 - 0
113,110,13,23/11/2025 16:15,Stade Francis-Le Blé,Stade Brestois 29,FC Metz,3 - 2
114,113,13,23/11/2025 16:15,Stade de la Beaujoire,FC Nantes,FC Lorient,1 - 1
115,117,13,23/11/2025 16:15,Stadium de Toulouse,Toulouse FC,Angers SCO,0 - 1
116,112,13,23/11/2025 19:45,Stade Pierre-Mauroy,LOSC Lille,Paris FC,4 - 2


## Priprema Podataka i df-a

In [5]:
#sjedinjavanje ta tri df niza podataka - po osi jedan - odnosno uzduž
df_BL = pd.concat([df_23_4, df_24_5, df_25_6], axis=0)
df_BL = df_BL.drop(["Match Number","Round Number","Date","Location"],axis=1)
df_BL.rename(columns={"Home Team":"H","Away Team":"A"},inplace=True)

rez_dom = df_BL["Result"].str.slice(0,1)
rez_gost = df_BL["Result"].str.slice(4,5)

df_rezultatski = pd.concat([df_BL,rez_dom,rez_gost],axis=1)
df_rezultatski.columns = ["H","A","Result","Rez1","Rez2"]

df1 = df_rezultatski

## Definisanje Funckije za Izracun Forme

In [6]:
def klub(tim):
    # Filtriramo DataFrame
    df = df1[(df1["H"] == tim) | (df1["A"] == tim)].copy()
    
    uslovi = [
    (df['H'] == tim) & (df['Rez1'] < df["Rez2"]),
    (df['A'] == tim) & (df['Rez1'] < df["Rez2"]),
    (df['H'] == tim) & (df['Rez1'] == df["Rez2"]),
    (df['A'] == tim) & (df['Rez1'] == df["Rez2"]),
    (df['H'] == tim) & (df['Rez1'] > df["Rez2"]),
    (df['A'] == tim) & (df['Rez1'] > df["Rez2"]),]
    
    vrijednosti = [-1, 1, 0, 0, 1, -1]
    df['Forma'] = np.select(uslovi, vrijednosti, default=0)
    df = df.drop(["Rez1","Rez2"],axis=1).tail(5)
    rezultat = df["Forma"].sum() / len(df["Forma"])
    return rezultat

## Provjera Rada te Funckije

In [7]:
klub('Paris FC')

-0.4

## Stvaranje Petlje za Automatski Izracun Forme Svih Klubova

In [8]:
#izvlacenje naziva svih klubova
timovi = pd.concat([df1["H"], df1["A"]]).unique()
timovi

array(['OGC Nice', 'Olympique de Marseille', 'Paris Saint-Germain',
       'Stade Brestois 29', 'Clermont Foot 63', 'Montpellier Hérault SC',
       'FC Nantes', 'Stade Rennais FC', 'RC Strasbourg Alsace', 'FC Metz',
       'Olympique Lyonnais', 'Toulouse FC', 'LOSC Lille',
       'Havre Athletic Club', 'FC Lorient', 'Stade de Reims', 'AS Monaco',
       'RC Lens', 'AJ Auxerre', 'Angers SCO', 'AS Saint-Étienne',
       'Paris FC'], dtype=object)

In [9]:
rezultati = {}

In [10]:
#petlja
for tim in timovi:
    rezultat = klub(tim)
    rezultati[tim] = rezultat

# Kreiranje DataFrame-a iz rezultata
df_forme = pd.DataFrame(list(rezultati.items()),columns = ["Klub","Rezultat"])
df_forme["Rezultat"] = df_forme["Rezultat"]

## Rezulat Forme za Sve

In [11]:
#Izracun
df_forme 

,Klub,Rezultat
0,OGC Nice,-0.2
1,Olympique de Marseille,0.4
2,Paris Saint-Germain,0.8
3,Stade Brestois 29,-0.4
4,Clermont Foot 63,-0.6
5,Montpellier Hérault SC,-0.8
6,FC Nantes,-0.2
7,Stade Rennais FC,0.4
8,RC Strasbourg Alsace,-0.2
9,FC Metz,0.2


In [12]:
#Ponovna provjera vrijednosti...
klub("Paris FC")

-0.4

In [13]:
df_forme.to_csv("forma_h.csv")

In [14]:
df_forme["Rezultat"] = df_forme["Rezultat"] * -1

In [15]:
df_forme


,Klub,Rezultat
0,OGC Nice,0.2
1,Olympique de Marseille,-0.4
2,Paris Saint-Germain,-0.8
3,Stade Brestois 29,0.4
4,Clermont Foot 63,0.6
5,Montpellier Hérault SC,0.8
6,FC Nantes,0.2
7,Stade Rennais FC,-0.4
8,RC Strasbourg Alsace,0.2
9,FC Metz,-0.2


In [16]:
df_forme.to_csv("forma_a.csv")